In [69]:
import requests
from bs4 import BeautifulSoup
import json
import re
import csv

In [70]:

url = "https://cymitquimica.com/categories/1828/nicotine-and-nicotine-derivatives/?srsltid=AfmBOor5CHkEY17td7i8alPNqfsjPX-VKsd6igxeoJFVzukYf576WD9_&page="
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

json_ld = None
for script in soup.find_all('script', type='application/ld+json'):
    try:
        json_ld = json.loads(script.string)
        break
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        continue

if json_ld:
    if isinstance(json_ld, dict):
        print("json_ld is a dictionary")
        print(json.dumps(json_ld, indent=2))
    else:
        print(f"Unexpected JSON-LD structure: {type(json_ld)}")
else:
    print("No valid JSON-LD found")


json_ld is a dictionary
{
  "@context": "http://schema.org",
  "@type": "WebSite",
  "url": "https://cymitquimica.com",
  "dateModified": "2019-02-15T00:00",
  "image": "https://static.cymitquimica.com/public/img/logo-cymit.png",
  "potentialAction": {
    "@type": "SearchAction",
    "target": "https://cymitquimica.com/search/{search_term_string}/",
    "query-input": "required name=search_term_string"
  },
  "sameAs": [
    "https://www.facebook.com/cymitquimica/",
    "https://twitter.com/cymitquimica",
    "https://es.linkedin.com/company/cymit-quimica-s-l-",
    "https://www.instagram.com/cymitquimica/",
    "https://www.pinterest.es/cymit/",
    "https://cymit.tumblr.com/"
  ]
}


In [71]:
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

product_links = soup.find_all("a", class_="js-product-link")

for link in product_links:
    product_url = link.get("href")
    if product_url:
        print(f"https://example.com{product_url}")


In [72]:
def scrape_page(page_number):
    url_new = url + str(page_number)
    response = requests.get(url_new)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        product_links = soup.find_all("a", class_="js-product-link")
        
        for link in product_links:
            product_url = link.get("href")
            if product_url:
                # Extract last part (after the last "/")
                last_part = product_url.strip("/").split("/")[-1]
                second_to_last_part = product_url.strip("/").split("/")[-2]
                product_urls.append(last_part)
                cas_numbers.append(second_to_last_part)
    else:
        print(f"Failed to retrieve page {page_number}")

In [ ]:
product_urls = []
cas_numbers = []

# for page in range(1, 14):
#     print(f"Scraping page {page}...")
#     scrape_page(page)

# csv_filename = "nicotine_derivatives_data.csv"
# with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
#     writer = csv.writer(file)
#     writer.writerow(["product_url", "cas_number"])
#     for url, cas in zip(product_urls, cas_numbers):
#         writer.writerow([url, cas])

# print(f"Number of entries saved in {csv_filename}: {len(product_urls)}")
# print(f"Number of CAS numbers: {len(cas_numbers)}")
def contains_uppercase(cas):
    return any(c.isupper() for c in cas)

for page in range(1, 14):
    print(f"Scraping page {page}...")
    scrape_page(page)

csv_filename = "nicotine_derivatives_data.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["product_url", "cas_number"])
    for url, cas in zip(product_urls, cas_numbers):
        # Only write the row if the CAS number doesn't contain an uppercase letter
        if not contains_uppercase(cas):
            writer.writerow([url, cas])

print(f"Number of entries saved in {csv_filename}: {len(product_urls)}")
print(f"Number of CAS numbers: {len(cas_numbers)}")

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Number of entries saved in nicotine_derivatives_data.csv: 252
Number of CAS numbers: 252


In [74]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
from tqdm import tqdm
import time
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--headless")  # Run in headless mode for speed
options.add_argument("--disable-gpu")  # Avoid GPU rendering issues
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)

# Set a page load timeout (e.g., 180 seconds)
driver.set_page_load_timeout(180)

In [ ]:
def get_smiles_from_cas(cas_number):
    url = f'https://www.1stsci.com/products/{cas_number}?cas={cas_number}'
    
    for attempt in range(3):
        try:
            driver.get(url)  # Load the page
            smiles_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "smiles"))
            )
            smiles_tag = driver.find_element(By.CLASS_NAME, 'smiles')
            smiles = smiles_tag.text
            print(f'{cas_number} gives the SMILES: {smiles} and is OK')
            # time.sleep(7)
            return smiles
        except Exception as e:
            print("Retrying CAS {cas_number}")
            time.sleep(7)
    print(f"FAILED TO RETRIEVE SMILES FOR CAS {cas_number} and is WRONGGGG.")
    return None

In [87]:
df = pd.read_csv('nicotine_derivatives_data.csv')
print(df.columns)

tqdm.pandas()
df['SMILES'] = df['cas_number'].progress_apply(get_smiles_from_cas)

total = len(df)
converted = df['SMILES'].notna().sum()
print(f"Total compounds: {total}")
print(f"Successfully converted: {converted}")
print(f"Conversion rate: {converted/total:.2%}")

df = df.dropna(subset=['SMILES'])
print(df)

df.to_csv('nicotine_derivatives_with_smiles.csv', index=False)

Index(['product_url', 'cas_number'], dtype='object')


  1%|          | 2/252 [00:09<20:20,  4.88s/it]

494-97-3 gives the SMILES: C1=CN=CC([C@@H]2CCCN2)=C1 and is OK


  1%|          | 3/252 [00:19<29:03,  7.00s/it]

485-35-8 gives the SMILES: O=C1C=CC=C2[C@H]3CNC[C@H](C3)CN21 and is OK


...

857146-29-3 gives the SMILES: CC1=CC(CO)=CN=C1C and is OK


100%|██████████| 252/252 [2:08:11<00:00, 28.76s/it]

20971-79-3 gives the SMILES: O=C1CCC(O1)c1cccnc1 and is OK
Retrying
Retrying
Retrying


100%|██████████| 252/252 [2:14:32<00:00, 32.03s/it]

FAILED TO RETRIEVE SMILES FOR CAS TR-B426320 and is WRONGGGG.
Total compounds: 252
Successfully converted: 208
Conversion rate: 82.54%
                                           product_url   cas_number  \
0                                    nornicotine-dl-rg     494-97-3   
1    15-methano-8h-pyrido12-a15diazocin-8-one-12345...     485-35-8   
2             3-pyridinecarboxamide-n-methoxy-n-methyl   95091-91-1   
3     4-iodo-1-methyl-12-dihydropyridine-3-carboxamide    6456-44-6   
4                  3-34-dihydro-2h-pyrrol-5-ylpyridine     532-12-7   
..                                                 ...          ...   
243                          n-4-methoxybenzylcotinine  887406-85-1   
244                         beta-nicotyrine-l-tartrate    4315-37-1   
247     5-methoxycarbonylnicotinic-acid-potassium-salt   27247-34-3   
249                5-hydroxymethyl-23-dimethylpyridine  857146-29-3   
250               rac-5-3-pyridyltetrahydro-2-furanone   20971-79-3   

            

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as GeoDataLoader
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from gnn import GNNModel, smiles_to_graph, MoleculeDataset

In [ ]:
df = pd.read_csv('nicotine_derivatives_with_smiles.csv')
df_cleaned = df[df['SMILES'].notna() & (df['SMILES'] != '')]
df_cleaned.to_csv('cleaned_nicotine_derivatives_with_smiles.csv', index=False)

In [ ]:
df = pd.read_csv('cleaned_nicotine_derivatives_with_smiles.csv')
nicotine_smiles = df['SMILES']

nicotine_graphs = [graph for graph in [smiles_to_graph(smile) for smile in nicotine_smiles] if graph is not None]

model_path = 'gnn_model.pth'
model = GNNModel(num_node_features=26, hidden_dim=128, output_dim=2)
model.load_state_dict(torch.load(model_path))
model.eval()

predictions = []

with torch.no_grad():
    for graph in nicotine_graphs:
        output = model(graph)
        _, predicted = torch.max(output, dim=1)
        predictions.append(predicted.item())

df['Predicted Label'] = predictions

df.to_csv('nicotine_derivatives_with_predictions.csv', index=False)

total = len(df)
predicted_count = len(predictions)
print(f"Total rows: {total}")
print(f"Predictions made: {predicted_count}")
print(f"Updated CSV saved with predictions.")

Error processing SMILES: nan - No registered converter was able to produce a C++ rvalue of type std::__1::basic_string<wchar_t, std::__1::char_traits<wchar_t>, std::__1::allocator<wchar_t>> from this Python object of type float
Error processing SMILES: nan - No registered converter was able to produce a C++ rvalue of type std::__1::basic_string<wchar_t, std::__1::char_traits<wchar_t>, std::__1::allocator<wchar_t>> from this Python object of type float
Error processing SMILES: nan - No registered converter was able to produce a C++ rvalue of type std::__1::basic_string<wchar_t, std::__1::char_traits<wchar_t>, std::__1::allocator<wchar_t>> from this Python object of type float
Error processing SMILES: nan - No registered converter was able to produce a C++ rvalue of type std::__1::basic_string<wchar_t, std::__1::char_traits<wchar_t>, std::__1::allocator<wchar_t>> from this Python object of type float
Error processing SMILES: nan - No registered converter was able to produce a C++ rvalue 

ValueError: Length of values (0) does not match length of index (260)